# load dependencies


In [59]:
import _resume_eval_import_helper

In [128]:
import os 
import tiktoken 
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv, find_dotenv
from uuid import uuid4
import pandas as pd
import json
from langchain_core.prompts import PromptTemplate
from prompts.resume_eval import RESUME_EVALUATION_PROMPT


from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI 
from langchain_anthropic import ChatAnthropic

from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from prompts.resume_eval import RESUME_EVALUATION_PROMPT

In [61]:
load_dotenv(find_dotenv("../../.env"))

True

# import dataset


In [62]:
data_path = "../repos/Resume-Screening-RAG-Pipeline/data/supplementary-data/"
synthetic_data_path = "../repos/Resume-Screening-RAG-Pipeline/data/main-data/"

job_description_path = data_path + "job_title_des.csv"
cleaned_resume_path = data_path + "cleaned_resume.csv"
synthetic_resume_path = synthetic_data_path + "synthetic-resumes.csv"

cleaned_resume = pd.read_csv(cleaned_resume_path)
synthetic_resumes = pd.read_csv(synthetic_resume_path)
job_description = pd.read_csv(job_description_path)

In [63]:
len(cleaned_resume), len(synthetic_resumes), len(job_description)

(162, 1000, 2277)

In [64]:
cleaned_resume["Category"].value_counts()

Category
Java Developer               13
Database                     11
Data Science                 10
Advocate                     10
DotNet Developer              7
Hadoop                        7
DevOps Engineer               7
Automation Testing            7
Testing                       7
Python Developer              6
Arts                          6
Health and fitness            6
Civil Engineer                6
HR                            6
SAP Developer                 6
Business Analyst              6
Electrical Engineering        5
Network Security Engineer     5
Mechanical Engineer           5
ETL Developer                 5
Blockchain                    5
Sales                         5
Operations Manager            4
Web Designing                 4
PMO                           3
Name: count, dtype: int64

In [65]:
cleaned_resume.columns, synthetic_resumes.columns, job_description.columns

(Index(['Category', 'Resume', 'ID'], dtype='object'),
 Index(['ID', 'Resume'], dtype='object'),
 Index(['Job Title', 'Job Description'], dtype='object'))

## helper functions


In [192]:
# estimating the cost 
def count_tokens(input_string: str) -> int:
  tokenizer = tiktoken.get_encoding("cl100k_base")
  tokens = tokenizer.encode(input_string)
  return len(tokens)

def calculate_cost(input_string: str, cost_per_million_tokens: float=5) -> float:
  num_tokens = count_tokens(input_string)
  total_cost = (num_tokens/1_000_000) * cost_per_million_tokens
  return total_cost

# import anthropic

# client = anthropic.Client()
# token_count = client.count_tokens(complete_template)
# print(token_count)
# generate unique id
def generate_unique_id(_):
    return str(uuid4())

# compare model outputs
def compare_model_outputs(model_results):
    """
    Compare original and recalibrated scores from different models and output a pretty table.
    
    :param model_results: Dict with model names as keys and JSON outputs as values
    :return: pandas DataFrame with a pretty table comparison
    """
    comparison_data = []
    
    score_types = [
        "technical_skills",
        "soft_skills",
        "required_experience",
        "qualifications"
    ]
    
    for model, result in model_results.items():
        # Ensure result is a dictionary
        if isinstance(result, str):
            result = json.loads(result)
        
        model_data = {"Model": model}
        
        # Extract original scores
        original_scores = result.get("resume_evaluation", {}).get("original_scores", {})
        
        # Extract recalibrated scores
        recalibrated_scores = result.get("recalibrated_scores", {})
        
        for score_type in score_types:
            model_data[f"original_{score_type}"] = original_scores.get(score_type, "N/A")
            model_data[f"recalibrated_{score_type}"] = recalibrated_scores.get(score_type, "N/A")
        
        # Add suitability
        model_data["suitability"] = result.get("assessment", {}).get("suitability", "N/A")
        
        comparison_data.append(model_data)
    
    # Create DataFrame
    df = pd.DataFrame(comparison_data)
    
    # Reorder columns
    column_order = ["Model"] + [f"{prefix}_{score_type}" for score_type in score_types for prefix in ["original", "recalibrated"]] + ["suitability"]
    df = df[column_order]
    
    return df

In [129]:
job_description["Job ID"] = job_description["Job Title"].apply(generate_unique_id)

In [137]:
jd = job_description[job_description["Job Title"]=="Machine Learning"]["Job Description"].values[0]
jd_id = job_description[job_description["Job Title"]=="Machine Learning"]["Job ID"].values[0]
cv = cleaned_resume.iloc[77]["Resume"]
cv_id = cleaned_resume.iloc[77]["ID"]

In [136]:
cleaned_resume[cleaned_resume["Category"] == "SAP Developer"]

,Category,Resume,ID
77,SAP Developer,Skills: ETL Data Warehousing SQL/PL SQL Ba...,3208b443-73ea-4236-bfdb-3f5d5d619347
78,SAP Developer,Competencies: SAP Business Intelligence Versio...,3e688d21-39ee-4601-9fc8-b74d9a359063
79,SAP Developer,Education Details \nJuly 2008 to February 2012...,ae358f9c-1fc5-46cc-a0c5-6eb78ccc5889
80,SAP Developer,Education Details \nMay 2013 Master Computer A...,a8b37267-f344-4cfc-8344-9aa920194e4e
81,SAP Developer,Education Details \nJanuary 2016 Bachelor Of E...,efe120ed-fc50-4fd1-8774-32cdfe61d663
82,SAP Developer,Education Details \nSAP Technical Architect \n...,954ab12a-1f45-46fb-b96c-4bb8d8f6d59f


# Agents


In [69]:

temperature = 0
max_tokens = 2048

groq_llm = ChatGroq(model="llama3-70b-8192", temperature=temperature0, max_tokens=max_tokens)
gpt_llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=temperature0, max_tokens=max_tokens)
anthropic_llm = ChatAnthropic(model="claude-3-5-sonnet-20240620", temperature=temperature, max_tokens=max_tokens)

resume_eval_prompt = PromptTemplate(
    input_variables=["job_description", "resume"],
    template=RESUME_EVALUATION_PROMPT
)

# Evaluate the resume


In [187]:
# categorize the resumes 
# we can sample the job pool to reduce the cost 
tech_jobs = ['Data Science', 
      'Java Developer', 'Business Analyst',
      'SAP Developer','Python Developer', 'DevOps Engineer',
      'Network Security Engineer','Database', 'Hadoop',
      'ETL Developer', 'DotNet Developer', 'Blockchain']

non_tech_jobs = [
  'HR', 'Advocate', 'Arts', 'Web Designing', 'Mechanical Engineer', 'Sales', 'Health and fitness',
  'Civil Engineer',  'Automation Testing', 'Electrical Engineering',
  'Operations Manager',  'PMO',
]

tech_pool = cleaned_resume[cleaned_resume["Category"].isin(tech_jobs)] # 88
non_tech_pool = cleaned_resume[cleaned_resume["Category"].isin(non_tech_jobs)] # 67

In [138]:
# llama3
groq_grader = resume_eval_prompt | groq_llm | JsonOutputParser()
groq_result = groq_grader.invoke({"job_description": jd, "resume": cv})

# gpt
gpt_grader = resume_eval_prompt | gpt_llm | JsonOutputParser()
gpt_result = gpt_grader.invoke({"job_description": jd, "resume": cv}) 

# claude
anthropic_grader = resume_eval_prompt | anthropic_llm | JsonOutputParser()
anthropic_result = anthropic_grader.invoke({"job_description": jd, "resume": cv})

In [142]:
model_results = {
    "groq": groq_result,
    "anthropic": anthropic_result,
    "gpt": gpt_result
}

comparison_df = compare_model_outputs(model_results)

# Display the pretty table
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
print(comparison_df.to_string(index=False))

# If you want to save it to a CSV file:
# comparison_df.to_csv("model_comparison.csv", index=False)

    Model  original_technical_skills  recalibrated_technical_skills  original_soft_skills  recalibrated_soft_skills  original_required_experience recalibrated_required_experience  original_qualifications  recalibrated_qualifications suitability
     groq                         20                             40                    40                        60                             0                              N/A                       60                           60          no
anthropic                         20                             30                    30                        40                             0                              N/A                      100                          100         kiv
      gpt                         14                             33                     0                         0                             0                              N/A                        0                            0          no


In [116]:
comparison_df

,Model,original_technical_skills,recalibrated_technical_skills,original_soft_skills,recalibrated_soft_skills,original_required_experience,recalibrated_required_experience,original_qualifications,recalibrated_qualifications,suitability
0,groq,80,90,70,80,100,N/A,100,100,yes
1,anthropic,80,85,60,70,100,N/A,100,100,yes
2,gpt,50,70,75,80,50,N/A,0,0,kiv


In [ ]:
output_dir = "../output/"

gpt_grader = resume_eval_prompt | gpt_llm | JsonOutputParser()
groq_grader = resume_eval_prompt | groq_llm | JsonOutputParser()
anthropic_grader = resume_eval_prompt | anthropic_llm | JsonOutputParser()


def evaluate_resume(job_description, resume, dob_id, cv_id):
  
  # llama3
  groq_result = groq_grader.invoke({"job_description": job_description, "resume": resume})

  # gpt
  gpt_result = gpt_grader.invoke({"job_description": job_description, "resume": resume}) 

  # claude
  anthropic_result = anthropic_grader.invoke({"job_description": job_description, "resume": resume})
  
  model_results = {
    "groq": groq_result,
    "anthropic": anthropic_result,
    "gpt": gpt_result
  }

  comparison_df = compare_model_outputs(model_results)
  
  if not os.path.exists(output_dir):
    os.makedirs(output_dir)

  comparison_df.to_csv(output_dir + f"{dob_id}_{cv_id}.csv", index=False)
  
  return comparison_df

evaluate_resume(jd, cv, jd_id, cv_id)